<h3>What is a Neural Network?</h3>
<p>To reiterate from the Neural Networks Learn Hub article, neural networks are a subset of machine learning, and they are at the heart of deep learning algorithms.<br> They are comprised of node layers, containing an input layer, one or more hidden layers, and an output layer. Each node connects to another and has an associated weight and threshold.<br> If the output of any individual node is above the specified threshold value, that node is activated, sending data to the next layer of the network.<br> Otherwise, no data is passed along to the next layer of the network.

While we primarily focused on feedforward networks in that article, there are various types of neural nets, which are used for different use cases and data types.<br>For example, recurrent neural networks are commonly used for natural language processing and speech recognition whereas convolutional neural networks (ConvNets or CNNs) are more often utilized for classification and computer vision tasks.<br> Prior to CNNs, manual, time-consuming feature extraction methods were used to identify objects in images.<br> However, convolutional neural networks now provide a more scalable approach to image classification and object recognition tasks, leveraging principles from linear algebra, specifically matrix multiplication, to identify patterns within an image.<br> That said, they can be computationally demanding, requiring graphical processing units (GPUs) to train models.</p>
<p>IBM, SOURCE <a href="https://www.ibm.com/topics/convolutional-neural-networks">HERE</p>

<h2>Topbots Graph Convolution Networks (GCN)</h2>
<p>Article Link <a href="https://www.topbots.com/graph-convolutional-networks/">HERE</p>

![image](00.images/topbots_00_graph_example.png "Example Graph")

Using this example graph we will build out the above matrices to observe how these features can be generated.

In [6]:
import numpy as np

In [7]:
n_nodes = 5

<h3>Generating Matrix A, the adjacency matrix</h3>
<p>This matrix will have a 1 for any given pair of nodes i,j that share an edge.<br> Both A<sub>(i,j)</sub> and A<sub>(j,i)</sub> will be set to 1, indicating the nodes are adjacent.</p>

In [8]:
matrix_a = np.zeros((n_nodes, n_nodes))

In [16]:
print(f'Matrix A, before populating values:\n{matrix_a}')

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 1. 1.]
 [0. 1. 1. 0. 1.]
 [1. 1. 1. 1. 0.]]


In [13]:
edges = [
    (0, 4),
    (1, 4),
    (2, 4),
    (3, 4),
    (1, 3),
    (2, 3)
]

In [14]:
for edge in edges:
    matrix_a[edge[0]][edge[1]] = 1
    matrix_a[edge[1]][edge[0]] = 1

In [15]:
print(f'Matrix A, after populating values:\n{matrix_a}')

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.]])

<h3>Generating Matrix D, the degree matrix</h3>
<p>This matrix is based off of the degree of a given node (aka, number of edges connected to that node)</p>

In [19]:
matrix_d = np.zeros((n_nodes, n_nodes))

In [21]:
print(f'Matrix D, before populating values:\n{matrix_d}')

Matrix D, before populating values:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [22]:
for edge in edges:
    matrix_d[edge[0]][edge[0]]+=1 
    matrix_d[edge[1]][edge[1]]+=1

In [24]:
print(f'Matrix D, after populating values:\n{matrix_d}')

Matrix D, after populating values:
[[1. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 4.]]


<h3>Generating Matrix X, the feature vector for each node</h3>
<p>This matrix will have be n_nodes x n_features in size.</p>

In [29]:
x_features = np.zeros((5, 3))

In [34]:
x_features[0] = np.array([-1.1, 3.2, 4.2])
x_features[1] = np.array([.4, 5.1, -1.2])
x_features[2] = np.array([1.2, 1.3, 2.1])
x_features[3] = np.array([1.4, -1.2, 2.5])
x_features[4] = np.array([1.4, 2.5, 4.5])

In [35]:
x_features

array([[-1.1,  3.2,  4.2],
       [ 0.4,  5.1, -1.2],
       [ 1.2,  1.3,  2.1],
       [ 1.4, -1.2,  2.5],
       [ 1.4,  2.5,  4.5]])

Feature values of neighbors by multiplying A * X

In [36]:
np.matmul(matrix_a, x_features)

array([[1.4, 2.5, 4.5],
       [2.8, 1.3, 7. ],
       [2.8, 1.3, 7. ],
       [3. , 8.9, 5.4],
       [1.9, 8.4, 7.6]])

<h3>Problem 01. Node Features</h3>
<p>We're still missing the features from the nodes themself.<br>
In order to get these we can add an identity matrix to A, to get a new adjacency matrix</p>

<blockquote>A<sub>f</sub> = A + &lambda; *I<sub>n</sub>
</blockquote>

<h2>Pytorch CSE224W Walkthrough</h2>
<p>Video Link <a href="https://www.youtube.com/watch?v=-UjytpbqX4A">HERE</p>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

In [2]:
batch_size = 32

## transformations
transform = transforms.Compose([transforms.ToTensor()])

## download the training set
trainset = torchvision.datasets.MNIST(root='./01.data', train=True,
                                      download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)

## download the testing set
testset = torchvision.datasets.MNIST(root='./01.data', train=False,
                                    download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                               shuffle=False, num_workers=2)



In [3]:
print(len(trainset))

60000


In [4]:
class MyModel(nn.Module):
    def  __init__(self):
        super(MyModel, self).__init__()

        # 28x28x1 => 26x26x32
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3)
        self.d1 = nn.Linear(26 * 26 * 32, 128)
        self.d2 = nn.Linear(128, 10)

    def forward(self, x):
        # 32x1x28x28 => 32x32x26x26
        x = self.conv1(x)
        x = F.relu(x)

        # flatten => 32 x (32*26*26)
        x = x.flatten(start_dim=1)
        #x = x.view(32, -1)

        # 32 x (32*26*26) => 32x128
        x = self.d1(x)
        x = F.relu(x)

        # logits => 32x10
        logits = self.d2(x)
        out = F.softmax(logits, dim=1)

        return out
    

<p>We train our model, outputting the accuracy along the way.<br>
We start by instantiating a model instance model, a loss function module criterion and optimizer optimizer, which will adjust the parameters of our model in order to minimize the loss output by criterion.
</p>

In [5]:
learning_rate = 0.001
num_epochs = 5

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = MyModel()

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
for epoch in range(num_epochs):
    train_running_loss = 0.0
    train_acc = 0.0

    ## training step
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)

        ## forward + backprop + loss
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()

        ## update model parameters
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += (torch.argmax(logits, 1).flatten() == labels).type(torch.float).mean().item()

    print('Epoch: %d | Loss: %.4f | Train Accuracy: %.2f' \
        % (epoch, train_running_loss / i, train_acc/i))


Epoch: 0 | Loss: 1.5547 | Train Accuracy: 0.91
Epoch: 1 | Loss: 1.4907 | Train Accuracy: 0.97
Epoch: 2 | Loss: 1.4819 | Train Accuracy: 0.98
Epoch: 3 | Loss: 1.4777 | Train Accuracy: 0.99
Epoch: 4 | Loss: 1.4752 | Train Accuracy: 0.99


In [20]:
test_acc = 0.0

for i, (images, labels) in enumerate(testloader, 0):
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    test_acc += (torch.argmax(outputs, 1).flatten() == labels).type(torch.float).mean().item()
    preds = torch.argmax(outputs, 1).flatten().cpu().numpy()
    l = labels.cpu().numpy()
    precision = metrics.precision_score(preds, l, average='micro')
    #print(f'Precision: {precision:.2f}')

print(f'Test Accuracy: {test_acc/i:.2f}')


Test Accuracy: 0.99


In [13]:
labels

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [14]:
preds

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

<h2>Setup for PyTorch Geometric</h2>

In [24]:
import time
from datetime import datetime

import networkx as nx
import numpy as np
import torch
import torch.optim as optim

from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader

import torch_geometric.transforms as T

from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'